In [1]:
#
import pandas as pd
df = pd.read_csv('unbalanced.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4174 entries, 0 to 4173
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             4174 non-null   object 
 1   Length          4174 non-null   float64
 2   Diameter        4174 non-null   float64
 3   Height          4174 non-null   float64
 4   Whole_weight    4174 non-null   float64
 5   Shucked_weight  4174 non-null   float64
 6   Viscera_weight  4174 non-null   float64
 7   Shell_weight    4174 non-null   float64
 8   Class           4174 non-null   object 
dtypes: float64(7), object(2)
memory usage: 293.6+ KB


In [2]:
#Transforming categorical columns
df['Class'] = df['Class'].map(lambda x: 0 if x == 'negative' else 1)
df = pd.get_dummies(df, columns=['Sex'], drop_first=True)
df

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Class,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,0,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,0,1,0
...,...,...,...,...,...,...,...,...,...,...
4169,0.560,0.430,0.155,0.8675,0.4000,0.1720,0.2290,0,0,1
4170,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,0,0,0
4171,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,0,0,1
4172,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,0,0,1


In [3]:
df['Class'].value_counts()

0    4142
1      32
Name: Class, dtype: int64

In [4]:
#Splitting training and test sets

from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.2, stratify=df['Class'], random_state=888)

features = df_train.drop(columns=['Class']).columns

In [9]:
df_train['Class'].value_counts()
df_test['Class'].value_counts()

0    3313
1      26
Name: Class, dtype: int64

In [7]:
#Oversampling
#Simple random oversampling

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=888)
X_resampled, y_resampled = ros.fit_resample(df_train[features], df_train['Class'])
y_resampled.value_counts()

0    3313
1    3313
Name: Class, dtype: int64

In [12]:
conda install -c conda-forge imbalanced-learn

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\rathakra2201\Anaconda3

  added / updated specs:
    - imbalanced-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2023.03  |           py39_0          66 KB
    anaconda-custom            |           py39_1           4 KB
    ca-certificates-2023.5.7   |       h56e8100_0         145 KB  conda-forge
    certifi-2023.5.7           |     pyhd8ed1ab_0         149 KB  conda-forge
    flask-1.1.2                |     pyh9f0ad1d_0          70 KB  conda-forge
    huggingface_hub-0.14.1     |     pyhd8ed1ab_0         156 KB  conda-forge
    imbalanced-learn-0.10.1    |     pyhd8ed1ab_0         131 KB  conda-forge
    jinja2-2.11.3              |     pyhd8ed1ab_2          94 KB  conda-forge
    libuv-1.44.2               |       h8ffe710_0         362 KB  conda-forge
    markupsa


The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/win-64::anaconda==2022.10=py39_0
  - defaults/win-64::anaconda-project==0.11.1=py39haa95532_0
  - defaults/win-64::bokeh==2.4.3=py39haa95532_0
  - defaults/win-64::conda-build==3.22.0=py39haa95532_0
  - defaults/noarch::conda-verify==3.4.2=py_1
  - defaults/noarch::cookiecutter==1.7.3=pyhd3eb1b0_0
  - defaults/win-64::dask==2022.7.0=py39haa95532_0
  - defaults/win-64::datashader==0.14.1=py39haa95532_0
  - defaults/win-64::distributed==2022.7.0=py39haa95532_0
  - defaults/win-64::holoviews==1.15.0=py39haa95532_0
  - defaults/win-64::hvplot==0.8.0=py39haa95532_0
  - defaults/noarch::intake==0.6.5=pyhd3eb1b0_0
  - defaults/noarch::ipywidgets==7.6.5=pyhd3eb1b0_1
  - defaults/noarch::jinja2-time==0.2.0=pyhd3eb1b0_3
  - defaults/win-64::jupyter==1.0.0=py39haa95532_8
  - defaults/win-64::jupyterlab==3.4.4=py39haa95532_0
  - defaults/noarch::jupyterlab_s

In [8]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.838962605548854

In [15]:
#Oversampling with shrinkage
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=888, shrinkage=0.1)
X_resampled, y_resampled = ros.fit_resample(df_train[features], df_train['Class'])

y_resampled.value_counts()

0    3313
1    3313
Name: Class, dtype: int64

In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.8059911540008041

In [17]:
#Oversampling using SMOTE (Synthetic Minority Over-sampling TEchnique)
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=888)
X_resampled, y_resampled = smote.fit_resample(df_train[features], df_train['Class'])

y_resampled.value_counts()

0    3313
1    3313
Name: Class, dtype: int64

In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.7913148371531966

In [19]:
#Undersampling
#Simple random undersampling
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(random_state=888)
X_resampled, y_resampled = rus.fit_resample(df_train[features], df_train['Class'])

y_resampled.value_counts()

0    26
1    26
Name: Class, dtype: int64

In [20]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.6465621230398071

In [21]:
#Undersampling using K-Means/Cluster Centroids
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=888)
X_resampled, y_resampled = cc.fit_resample(df_train[features], df_train['Class'])

y_resampled.value_counts()

0    26
1    26
Name: Class, dtype: int64

In [22]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.6174105347808605

In [23]:
#Undersampling using Tomek links
from imblearn.under_sampling import TomekLinks

tl = TomekLinks()
X_resampled, y_resampled = tl.fit_resample(df_train[features], df_train['Class'])

y_resampled.value_counts()

0    3298
1      26
Name: Class, dtype: int64

In [24]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.683956574185766

In [25]:
#Combining Oversampling and Undersampling
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=888)
X_resampled, y_resampled = smote_tomek.fit_resample(df_train[features], df_train['Class'])

y_resampled.value_counts()

0    3309
1    3309
Name: Class, dtype: int64

In [26]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=888)

clf.fit(X_resampled, y_resampled)
y_pred = clf.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.7913148371531966

In [27]:
#Weighing classes differently
from sklearn.utils import compute_class_weight

weights = compute_class_weight('balanced', classes=df_train['Class'].unique(), y=df_train['Class'])
weights

array([ 0.50392394, 64.21153846])

In [28]:
print((df_train['Class'] == 0).sum()*weights[0])

print((df_train['Class'] == 1).sum()*weights[1])

1669.5
1669.5000000000002


In [29]:
print((df_train['Class'] == 0).sum()*weights[0] + (df_train['Class'] == 1).sum()*weights[1])

print((df_train['Class'] == 0).sum() + (df_train['Class'] == 1).sum())

3339.0
3339


In [30]:
from sklearn.linear_model import LogisticRegression

clf_weighted = LogisticRegression(class_weight='balanced', random_state=888)
clf_weighted.fit(df_train[features], df_train['Class'])

y_pred = clf_weighted.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.8275030156815439

In [31]:
from sklearn.linear_model import LogisticRegression

clf_weighted = LogisticRegression(class_weight={0: 1, 1: 100}, random_state=888)

clf_weighted.fit(df_train[features], df_train['Class'])
y_pred = clf_weighted.predict_proba(df_test[features])[:, 1]

from sklearn.metrics import roc_auc_score
roc_auc_score(df_test['Class'], y_pred)

0.8375552874949739